# What is pytorch geometric?
Pytorch geometric is a library built on top of pytorch that aims to simplify the implementation of geometric deep learning models. First some motivation as to what geometric deep learning is.

## Classical deep learning on graph-based data
The naive approach to this problem is to find the adjacency matrix of a input graph and use that as input to a neural network as there are already several successful architectures that take matrices as input, primarily those related to computer vision (ex. convolutional neural networks, feed forward neural networks, ...). However, this approach has some problems:
- Model cannot adapt to changes in graph size (I.E number of nodes). 
- The adjacency matrix is not permutation invariant (I.E one graph might have multiple distinct adjacency matrices).
In many cases, not only is the structure of the graph important but also the properties of the nodes in a particular graph. Thus, in a general case, a network must also account for a secondary features matrix containing node related information.

## Graph Neural Networks

### The computation graph of a node
Every node of a graph has a computation graph. The computation graph of a node is a tree where the root node is the node in question and it's children are it's neighbours. The neighbours in the child layer also have children (composed of the neighbours's neighbours). This process continuous until all nodes are exhuasted. **Node repetition is valid.**

Sometimes, the process of generating the this computation graph layer by layer is informally referred to as unrolling the graph about a particular node. Often unrolling is limited to a particular number, this imposes a constraint on the depth of each computation graph.

### Action on the computation graph of a node
GNNs process graph-data using two mechanisms: aggregation and propagation. Starting from the leaf-nodes, data is supplied from the node-feature matrix. If multiple children share a parent, their information is combined through aggregation. The aggregated product is then transmitted to the parent, considering the parent's feature vector, via the propagation mechanism. This process continues until information reaches the root node.

An important property of the aggregation mechanism is its permutation invariance, meaning it doesn't depend on the order of inputs supplied.

### How much to unroll
Unrolling too little can result in important information not being transmitted whereas unrolling too much makes computation far more expensive even though the information transmitted from distant nodes might not affect the overall result.

### Computation formally described for a particular case
The above computation can be consicely described by the following equations:
$$
\begin{gather}
H_v^0 = X_v \\ 
h_v^{k+1} = \sigma\bigg(W_k\sum_{u\in N(u)}\frac{h_u^k}{|N(v)|} + B_kh_v^k\bigg) \\ 
Z_v = h_v^K
\end{gather}
$$
where, $\sigma$ is an activation function $W_k$ and $B_k$ are trainable weights, $N(u)$ is the set of neighbours of node $u$, and, $\sum$ is some permutation invariant aggregation function. Keep in mind that this equation only comes after making a specific selection as to how the propagation mechanism acts.

#  Graph SAGE

Graph SAGE is a architecture that is mostly the same as the general GNN described above with the exception of the propagation and aggregation mechanism, which we describe here,
$$h_v^{k+1} = \sigma\bigg(\bigg[W_k\cdot AGG\bigg(\{h_u^{k-1}, \forall u\in N(v)\}\bigg), B_kh_v^k\bigg]\bigg)$$
where, $[,]$ is a concatenation and $AGG$ is a general aggregation strategy.

In [2]:
# Tutorial - I code from this point on
import torch_geometric
from torch_geometric.datasets import Planetoid

In [3]:
dataset = Planetoid(root="tutorial1", name="Cora")

In [4]:
print(dataset)
print("number of graphs:\t\t", len(dataset))
print("number of classes:\t\t", dataset.num_classes)
print("number of node features:\t", dataset.num_node_features)
print("number of edge features:\t", dataset.num_edge_features)

Cora()
number of graphs:		 1
number of classes:		 7
number of node features:	 1433
number of edge features:	 0


In [5]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [6]:
data = dataset[0]

In [7]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv = SAGEConv(dataset.num_features,
                            dataset.num_classes,
                            aggr = "max")
        
    def forward(self):
        x = self.conv(data.x, data.edge_index)
        return F.log_softmax(x, dim=1)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [20]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()

def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


In [23]:
best_val_acc = test_acc = 0
for epoch in range(1, 100):
    train()
    _, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Val: {:.4f}, Test: {:.4f}'
    
    if epoch % 10 == 0:
        print(log.format(epoch, best_val_acc, test_acc))

Epoch: 010, Val: 0.7120, Test: 0.7340
Epoch: 020, Val: 0.7120, Test: 0.7340
Epoch: 030, Val: 0.7120, Test: 0.7340
Epoch: 040, Val: 0.7120, Test: 0.7340
Epoch: 050, Val: 0.7120, Test: 0.7340
Epoch: 060, Val: 0.7120, Test: 0.7340
Epoch: 070, Val: 0.7120, Test: 0.7340
Epoch: 080, Val: 0.7120, Test: 0.7340
Epoch: 090, Val: 0.7120, Test: 0.7340


# References
1. [ Pytorch Geometric tutorial: Introduction to Pytorch geometric ](https://youtu.be/JtDgmmQ60x8)
2. [Graph SAGE paper](https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf)